In [63]:
import pandas as pd
from pandarallel import pandarallel
import ast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [64]:
from gensim.models import Word2Vec

In [65]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [66]:
dataRew=pd.read_csv('../Dataset/datiClean.csv')
dataMovie=pd.read_csv('../Dataset/movieclean.csv')

In [67]:
dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)

In [68]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(ast.literal_eval)

In [69]:
dataRew.drop(['review_text'],axis=1,inplace=True)

In [70]:
dataMovie.drop(['plot_synopsis','plot_summary'],axis=1,inplace=True)

### Split the Dataset

important distribute well the labels in the train and test dataset

In [71]:
dataRew.drop(['review_date','movie_id','user_id','rating','review_summary'],axis=1,inplace=True)

In [72]:
x=dataRew['clean_review']
y=dataRew['is_spoiler']

In [73]:
## Stratify bilancia i dataset 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,random_state=42)

In [74]:
y_train.value_counts()

is_spoiler
False    338391
True     120739
Name: count, dtype: int64

In [75]:
y_test.value_counts()

is_spoiler
False    84598
True     30185
Name: count, dtype: int64

### Apply Logistic Regression and Naive Bayes, with k fold

### Function for computing the result of the model

In [76]:
def print_mean():
    # Calcola le medie delle metriche
    mean_accuracy = np.mean(metrics['accuracy'])
    mean_precision = np.mean(metrics['precision'])
    mean_recall = np.mean(metrics['recall'])
    mean_f1_score = np.mean(metrics['f1_score'])

    # Stampa le medie delle metriche
    print("Mean Accuracy:", mean_accuracy)
    print("Mean Precision:", mean_precision)
    print("Mean Recall:", mean_recall)
    print("Mean F1 Score:", mean_f1_score)

In [77]:
def print_test(y_pred_test):
    # Calcolo delle metriche di valutazione sul set di test
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)

    # Stampa delle metriche di valutazione sul set di test
    print("Test Accuracy:", accuracy_test)
    print("Test Precision:", precision_test)
    print("Test Recall:", recall_test)
    print("Test F1 Score:", f1_score_test)

In [78]:
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
def computeLogistic(folds,iter,X,y_train):
    logistic_reg=LogisticRegression(max_iter=iter)
    ## Stratified k-fold grant me a balance division of classes

    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(X, y_train):
        X_fold_train, X_fold_val = X[train_index], X[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Training
        logistic_reg.fit(X_fold_train, y_fold_train)
        
        # Validation
        y_pred = logistic_reg.predict(X_fold_val)
        
        # Compute metrics
        metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        metrics['precision'].append(precision_score(y_fold_val, y_pred))
        metrics['recall'].append(recall_score(y_fold_val, y_pred))
        metrics['f1_score'].append(f1_score(y_fold_val, y_pred))
    return logistic_reg

In [79]:
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
def compute_naive(folds,X,y_train):
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    naive_bayes = MultinomialNB()
    
    for train_index, val_index in kf.split(X, y_train):
        X_fold_train, X_fold_val = X[train_index], X[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Training
        naive_bayes.fit(X_fold_train, y_fold_train)
        
        # Validation
        y_pred = naive_bayes.predict(X_fold_val)
        
        # Compute metrics
        metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        metrics['precision'].append(precision_score(y_fold_val, y_pred))
        metrics['recall'].append(recall_score(y_fold_val, y_pred))
        metrics['f1_score'].append(f1_score(y_fold_val, y_pred))
    return naive_bayes

### Using Bag of Words

first create the text, using the array of vector

In [80]:
text=[" ".join(word) for word in X_train]

In [81]:
textT=[" ".join(word) for word in X_test]

In [82]:
## Bag of Words for train
vect=CountVectorizer()
X=vect.fit_transform(text)

In [83]:
## Bag of Words for test
X_t=vect.transform(textT)

In [84]:
y_train=y_train.values
y_test=y_test.values

## Logistic Regression

### Result Train and Validation

In [85]:
logistic_reg=computeLogistic(5,1000,X,y_train)
print_mean()

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Mean Accuracy: 0.7637357611134102
Mean Precision: 0.5801693331064446
Mean Recall: 0.36755316751267
Mean F1 Score: 0.4500068373700392


### Result Test

In [86]:
y_pred_test = logistic_reg.predict(X_t)
print_test(y_pred_test)


Test Accuracy: 0.7650784523840638
Test Precision: 0.5838716399249844
Test Recall: 0.37131025343713764
Test F1 Score: 0.45393977440716066


## Naive Bayes

### Result Validation, Training

In [87]:
naive_bayes=compute_naive(5,X,y_train)
print_mean()

Mean Accuracy: 0.7513645372770239
Mean Precision: 0.542012190226676
Mean Recall: 0.4287968419357667
Mean F1 Score: 0.4734264620712711


### Result Test

In [88]:
y_pred_test = naive_bayes.predict(X_t)

print_test(y_pred_test)

Test Accuracy: 0.7410853523605412
Test Precision: 0.5081326352530541
Test Recall: 0.4822925294020209
Test F1 Score: 0.49487549927764085


## Tf-idf

In [89]:

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(text)
X_test_tfidf = tfidf_vectorizer.transform(textT)

In [90]:
X_train_tfidf

<459130x240545 sparse matrix of type '<class 'numpy.float64'>'
	with 47974839 stored elements in Compressed Sparse Row format>

### Logistic Regression

In [91]:
logistic_reg=computeLogistic(5,1000,X_train_tfidf,y_train)
print_mean()

Mean Accuracy: 0.7607191862871081
Mean Precision: 0.5807758989773191
Mean Recall: 0.3976235387407522
Mean F1 Score: 0.4637083095289595


In [92]:
y_pred_test = logistic_reg.predict(X_test_tfidf)
print_test(y_pred_test)

Test Accuracy: 0.7794795396530845
Test Precision: 0.6543358459491987
Test Recall: 0.34222295842305783
Test F1 Score: 0.44940398503436874


### Naive Bayes

In [93]:
naive_bayes=compute_naive(5,X_train_tfidf,y_train)
print_mean()

Mean Accuracy: 0.7556879315226623
Mean Precision: 0.6143629973470797
Mean Recall: 0.3038579192685539
Mean F1 Score: 0.3587166658453411


In [94]:
y_pred_test = naive_bayes.predict(X_test_tfidf)

print_test(y_pred_test)

Test Accuracy: 0.7404406575886673
Test Precision: 0.7848837209302325
Test Recall: 0.017889680304787145
Test F1 Score: 0.03498202312700418


### Word2Vec

In [10]:
X_preWord=dataRew["clean_review"]

In [11]:
X_preWord

0         [oscar, year, shawshank, redemption, written, ...
1         [shawshank, redemption, without, doubt, one, b...
2         [believe, film, best, story, ever, told, film,...
3         [yes, spoiler, film, emotional, impact, find, ...
4         [heart, extraordinary, movie, brilliant, indel...
                                ...                        
573908    [go, wise, fast, pure, entertainment, assembli...
573909    [well, shall, say, one, fun, rate, three, plot...
573910    [go, best, movie, ever, seen, seen, lot, movie...
573911    [call, 1999, teenage, version, pulp, fiction, ...
573912    [movie, made, doubt, sucker, family, rebelling...
Name: clean_review, Length: 573913, dtype: object

In [12]:
word2vec_model = Word2Vec(X_preWord, vector_size=100, window=5, min_count=1, sg=1)

In [13]:
words=word2vec_model.wv.most_similar("spoiler")

In [14]:
words

[('alert', 0.9097859859466553),
 ('warning', 0.8329949975013733),
 ('aheadi', 0.8238783478736877),
 ('alerti', 0.8205029368400574),
 ('aheadthe', 0.8145661354064941),
 ('spoilersthis', 0.8143439292907715),
 ('spolier', 0.8070707321166992),
 ('spoilersthe', 0.8060628771781921),
 ('spoliers', 0.80155348777771),
 ('withinthis', 0.7661573886871338)]

In [35]:
X_word2vectrain = [np.mean([word2vec_model.wv[word] for word in text], axis=0) for text in X_preWord]

In [36]:
X_train, X_test,y_train, y_test  = train_test_split(X_word2vectrain,y, test_size=0.2, random_state=42,stratify=y)

In [50]:
y_train=np.array(y_train)

In [37]:
y_train.value_counts()

is_spoiler
False    338391
True     120739
Name: count, dtype: int64

In [47]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [62]:
logistic_reg=computeLogistic(5,1000,X_train,y_train)
print_mean()

Mean Accuracy: 0.748494616630003
Mean Precision: 0.39793625061490867
Mean Recall: 0.13431175467082626
Mean F1 Score: 0.20083584460272322


In [55]:
y_pred_test = logistic_reg.predict(X_test)
print_test(y_pred_test)

Test Accuracy: 0.7535087948563812
Test Precision: 0.5910841517427305
Test Recall: 0.20337916183534868
Test F1 Score: 0.3026299573587045


## Naive Bayes

### Transform with normalization
i dati devono essere normalizzati perchè essendo negativi, non possiamo applicare il naive bayes

In [57]:
from sklearn.preprocessing import MinMaxScaler

In [58]:
scaler = MinMaxScaler()
X_train_word2vec_normalized = scaler.fit_transform(X_train)
X_test_word2vec_normalized = scaler.transform(X_test)

In [60]:
naive_bayes=compute_naive(5,X_train_word2vec_normalized,y_train)
print_mean()

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Prec

Mean Accuracy: 0.7456276000261364
Mean Precision: 0.29845218796118145
Mean Recall: 0.1007338160031197
Mean F1 Score: 0.1506268834520424


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [61]:
y_pred_test = naive_bayes.predict(X_test_word2vec_normalized)

print_test(y_pred_test)

Test Accuracy: 0.7370255177160381
Test Precision: 0.0
Test Recall: 0.0
Test F1 Score: 0.0


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
